In [2]:
import pandas as pd
from pandas import read_excel
from os import listdir

In [4]:
list_files = listdir("ods") #Nom des fichiers ods 

dico_files est un dictionnaire qui contient des dataframe

In [5]:
dico_files = {} #Clé = Nom du fichier, Valeur = dataframe
for file in list_files:
    dico_files[file] = read_excel(f"ods/{file}")
print(list_files)


['leg22_T1.ods', 'leg22_T2.ods', 'leg24_T1.ods', 'leg24_T2.ods', 'mun20_T1.ods', 'mun20_T2.ods', 'pre22_T1.ods']


Créer pour chaque dataframe une colonne election, année et tour dans le but de tous les merge à la fin tout en gardant la trace de quel résultat appartient à quelle élection/tour

In [6]:
i = 0
for key in dico_files.keys():
    dico_files[key]["election"] = list_files[i][0:3]
    dico_files[key]["année"] = list_files[i][3:5]
    dico_files[key]["tour"] = list_files[i][7]
    dico_files[key] = dico_files[key].drop(columns = 'Circonscription')
    i += 1

Corriger des erreurs de typo présentes dans les données 

In [7]:
dico_files["mun20_T1.ods"] = dico_files["mun20_T1.ods"].rename(columns = {'Exprimes':'Exprimés'})
dico_files["mun20_T2.ods"] = dico_files["mun20_T1.ods"].rename(columns = {'Exprimes':'Exprimés'})
dico_files["leg24_T2.ods"] = dico_files["leg24_T2.ods"].rename(columns = {'Absentions':'Abstentions'})

In [8]:
dico_files.keys()

dict_keys(['leg22_T1.ods', 'leg22_T2.ods', 'leg24_T1.ods', 'leg24_T2.ods', 'mun20_T1.ods', 'mun20_T2.ods', 'pre22_T1.ods'])

### Valeurs manquantes

Il manquait des valeurs pour la ligne d'indice 134 que je suis allée chercher sur internet 

In [9]:
valeurs_de_remplacement = {'Inscrits': 1391, 'Abstentions': 878, "Votants":513, "Exprimés":501, "ED":25, "DROITE":154, "GAUCHE": 295, "EG":17, "DIVERS":10}  # Dictionnaire des valeurs à remplir
for col, valeur in valeurs_de_remplacement.items():
    dico_files["pre22_T1.ods"].loc[134, col] = valeur

### Merge tout mes dataframes 

### Traitements

Je vais merge tout les fichiers csv dans un seul et même dataframe 

In [10]:
df = pd.DataFrame()
for key in dico_files.keys():
    df = pd.concat([df ,dico_files[key]])

Je sais qu'il ne manque aucun quartiers de mairiedonc je retire toutes les lignes qui n'ont pas de quartiers de mairie

In [11]:
masque_nettoyage = df.loc[df["Quartiers Mairie"].isna() == True]

In [12]:
df = df.drop(index = masque_nettoyage.index)

J'ai encore des valeurs manquantes, et c'est normal, il y a généralement pas de partis d'extrème gauche ou divers en 2nd tour des élections. 

In [13]:
df["Participation"] = df["Exprimés"]/df["Inscrits"]

In [14]:
df.to_csv("dataset.csv")